In [1]:
#Load libraries
import pandas as pd
import numpy as np

In [2]:
#Read data to DataFrame
df = pd.read_json('currencies.json')

In [3]:
#Let's get a sense of the data
df.head()

,date,day_close,day_high,day_low,day_market_cap,day_open,day_volume,long_name,market_rank,short_name
0,2019-04-09,0.000049,0.000049,0.000037,"1,296,018",0.000042,225,Elite,#1262,(1337)
1,2019-04-08,0.000042,0.000046,0.000039,"1,108,819",0.000043,162,Elite,#1262,(1337)
2,2019-04-09,0.331090,0.335818,0.259410,-,0.262610,"18,090",1irstcoin,#1977,(FST)
3,2019-04-09,0.725394,0.733987,0.717767,-,0.725590,"7,063",1SG,#2009,(1SG)
4,2019-04-09,0.177210,0.204110,0.160943,-,0.177008,"4,125",COIN,#2023,(1X2)


In [4]:
#DataFrame structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537630 entries, 0 to 537629
Data columns (total 10 columns):
date              537630 non-null datetime64[ns]
day_close         537630 non-null object
day_high          537630 non-null object
day_low           537630 non-null object
day_market_cap    537630 non-null object
day_open          537630 non-null object
day_volume        537630 non-null object
long_name         537630 non-null object
market_rank       537630 non-null object
short_name        537630 non-null object
dtypes: datetime64[ns](1), object(9)
memory usage: 41.0+ MB


In [5]:
#Clean market rank variable
df['market_rank'] = (
df['market_rank']
    .str.replace('#','')
    .astype(int)
)

#Clean short name variable
df['short_name'] = (
df['short_name']
    .str.replace('(','')
    .str.replace(')','')
)

#Clean day close variable
df['day_close'] =  (
df['day_close']
    .str.replace(',','')
    .astype(float)
)
#Check results
df.head()

,date,day_close,day_high,day_low,day_market_cap,day_open,day_volume,long_name,market_rank,short_name
0,2019-04-09,0.000049,0.000049,0.000037,"1,296,018",0.000042,225,Elite,1262,1337
1,2019-04-08,0.000042,0.000046,0.000039,"1,108,819",0.000043,162,Elite,1262,1337
2,2019-04-09,0.331090,0.335818,0.259410,-,0.262610,"18,090",1irstcoin,1977,FST
3,2019-04-09,0.725394,0.733987,0.717767,-,0.725590,"7,063",1SG,2009,1SG
4,2019-04-09,0.177210,0.204110,0.160943,-,0.177008,"4,125",COIN,2023,1X2


In [11]:
#Sort DataFrame
df.sort_values(['market_rank', 'date'], ascending = [True, False], inplace = True)
df.head()

,date,day_close,day_high,day_low,day_market_cap,day_open,day_volume,long_name,market_rank,short_name,close_diff
352561,2019-04-09,0.000103,0.000157,0.000052,"3,361,593",0.000106,210,Pandacoin,576,PND,NaN
352571,2019-04-08,0.000106,0.000158,0.000101,"3,436,161",0.000101,"1,437",Pandacoin,576,PND,0.000003
352581,2019-04-07,0.000101,0.000154,0.000100,"3,296,951",0.000101,117,Pandacoin,576,PND,-0.000005
352591,2019-04-06,0.000101,0.000152,0.000100,"3,299,026",0.000103,177,Pandacoin,576,PND,0.000000
352601,2019-04-05,0.000103,0.000151,0.000098,"3,347,236",0.000098,233,Pandacoin,576,PND,0.000002


In [7]:
#Take first differnces
df['close_diff'] = df.groupby(['short_name'])['day_close'].transform(lambda x: x.diff())
df.head()

,date,day_close,day_high,day_low,day_market_cap,day_open,day_volume,long_name,market_rank,short_name,close_diff
532691,2019-04-09,0.039118,0.039295,0.012675,"76,143",0.012963,0,ZoZoCoin,1558,ZZC,NaN
532701,2019-04-08,0.012963,0.016389,0.012916,"25,232",0.013727,5,ZoZoCoin,1558,ZZC,-0.026155
532710,2019-04-07,0.013724,0.022234,0.012722,"26,714",0.012732,65,ZoZoCoin,1558,ZZC,0.000761
532711,2019-04-06,0.012745,0.019660,0.012598,"24,808",0.014760,12,ZoZoCoin,1558,ZZC,-0.000979
532720,2019-04-05,0.014756,0.075004,0.014709,"28,723",0.015945,13,ZoZoCoin,1558,ZZC,0.002011


In [10]:
#Create example df
ethereum_df = df.loc[df['short_name'] == 'ETH']
ethereum_df.head()

,date,day_close,day_high,day_low,day_market_cap,day_open,day_volume,long_name,market_rank,short_name,close_diff
